# Распознавание предметов одежды при помощи полносвязной нейронной сети

In [12]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

Загрузим набор данных FashionMNIST:

In [13]:
import torchvision.datasets
MNIST_train = torchvision.datasets.MNIST('./', download=True, train=True)
MNIST_test = torchvision.datasets.MNIST('./', download=True, train=False)

Разделим данные на тестовые и тренировочные множества:

In [14]:
X_train = MNIST_train.train_data
y_train = MNIST_train.train_labels
X_test = MNIST_test.test_data
y_test = MNIST_test.test_labels

D:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
D:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
D:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
D:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Посмотрим на тип данных:

In [15]:
X_train.dtype, y_train.dtype

(torch.uint8, torch.int64)

Переведём их в тип float:

In [16]:
X_train = X_train.float()
X_test = X_test.float()

Узнаем размерности получившихся четырёх множеств:

In [17]:
X_train.shape, X_test.shape
y_train.shape, y_test.shape

(torch.Size([60000]), torch.Size([10000]))

Выведем на экран одно из изображений:

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0, :, :])
plt.show()
print(y_train[0])

Изменим размерности, чтобы выполнялось последующее матричное умножение:

In [18]:
X_train = X_train.reshape([-1, 28 * 28])
X_test = X_test.reshape([-1, 28 * 28])

Напишем класс для нейронной сети, она будет состоять из входного слоя с последующей функцией активации сигмоидой и входного слоя:

In [19]:
class FashionMNISTNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(FashionMNISTNet, self).__init__()
        self.fc1 = torch.nn.Linear(28 * 28, n_hidden_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 10)

    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        return x


Создадим экземпляр класса с 90 нейронами на скрытом слое:

In [20]:
fashion_mnist_net = FashionMNISTNet(90)

Определим функцию потерь через кросс-энтропию, а в качестве оптимизатора возьмём Adam с шагом обучения 0,001.

In [22]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fashion_mnist_net.parameters(), lr=1.0e-3)

Обучим нашу нейронную сеть:

In [23]:
batch_size = 100

test_accuracy_history = []
test_loss_history = []

for epoch in range(100):
    order = np.random.permutation(len(X_train))
    
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = fashion_mnist_net.forward(X_batch)
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()

        
        optimizer.step()


    test_preds = fashion_mnist_net.forward(X_test)
    
    test_loss_history.append(loss(test_preds, y_test))
    
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    test_accuracy_history.append(accuracy)
    print(accuracy)

tensor(0.9063)
tensor(0.9189)
tensor(0.9214)
tensor(0.9208)
tensor(0.9239)
tensor(0.9260)
tensor(0.9281)
tensor(0.9304)
tensor(0.9360)
tensor(0.9230)
tensor(0.9308)
tensor(0.9360)
tensor(0.9356)
tensor(0.9309)
tensor(0.9371)
tensor(0.9327)
tensor(0.9384)
tensor(0.9392)
tensor(0.9317)
tensor(0.9363)
tensor(0.9371)
tensor(0.9392)
tensor(0.9383)
tensor(0.9398)
tensor(0.9419)
tensor(0.9440)
tensor(0.9414)
tensor(0.9441)
tensor(0.9416)
tensor(0.9451)
tensor(0.9437)
tensor(0.9416)
tensor(0.9446)
tensor(0.9476)
tensor(0.9482)
tensor(0.9409)
tensor(0.9424)
tensor(0.9449)
tensor(0.9416)
tensor(0.9505)
tensor(0.9428)
tensor(0.9450)
tensor(0.9465)
tensor(0.9414)
tensor(0.9465)
tensor(0.9484)
tensor(0.9489)
tensor(0.9461)
tensor(0.9489)
tensor(0.9483)
tensor(0.9496)
tensor(0.9488)
tensor(0.9464)
tensor(0.9512)
tensor(0.9513)
tensor(0.9518)
tensor(0.9526)
tensor(0.9490)
tensor(0.9528)
tensor(0.9506)
tensor(0.9500)
tensor(0.9519)
tensor(0.9525)
tensor(0.9480)
tensor(0.9512)
tensor(0.9516)
tensor(0.9

Выведем на экран значения точности и потерь:

In [ ]:
plt.plot(test_accuracy_history)
plt.plot(test_loss_history)